In [1]:
from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field
import requests
import datetime


class RetornaTempArgs(BaseModel):
    latitude: float = Field(
        description="Latitude da localidade para a qual deseja obter a temperatura"
    )
    longitude: float = Field(
        description="Longitude da localidade para a qual deseja obter a temperatura"
    )

@tool(args_schema=RetornaTempArgs)
def retorna_temperatura(latitude:float, longitude:float):
    """Retorna a temperatura atual para um dada localidade"""

    URL = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()

        hora_atual = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado["hourly"]["time"]]
        index_prox = min(range(len(lista_horas)), key=lambda i: abs(lista_horas[i] - hora_atual))

        temp_atual = resultado["hourly"]["temperature_2m"][index_prox]
        return temp_atual

    else:
        raise Exception(f"Erro ao obter a temperatura na API {URL}: {resposta.status_code}")


import wikipedia
wikipedia.set_lang("pt")

query = "Isaac Asimov"

@tool
def busca_wikipedia(query:str):
    """Faz busca no wikipedia e retorna resumos de paginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []

    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f"Titulo da pagina: {titulo} \n Resumo: {wiki_page.summary}")
        except:
            pass
    if not resumos:
        return "Nenhuma informação encontrada"
    else:
        return "\n\n".join(resumos)

c:\Users\johns\OneDrive\Documentos\Projetos\GitHub\Regdoor_Project\venv\Lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
